In [1]:

!pip install kagglehub tensorflow opencv-python scikit-learn matplotlib


In [2]:

import kagglehub
import os

# Download dataset
path = kagglehub.dataset_download("gti-upm/leapgestrecog")
data_dir = os.path.join(path, "leapGestRecog")
print("Downloaded and extracted to:", data_dir)


Downloaded and extracted to: C:\Users\Mamba\.cache\kagglehub\datasets\gti-upm\leapgestrecog\versions\1\leapGestRecog


In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

def find_dataset_folder(base_path):
    for root, dirs, files in os.walk(base_path):
        for dir in dirs:
            if "leapGestRecog" in dir:
                return os.path.join(root, dir)
    return base_path  # fallback

# Define the path variable before using it
path = "."  # Replace with the actual base path where you expect to find the dataset
data_dir = find_dataset_folder(path)
print("Using data directory:", data_dir)

def load_data(image_size=(64, 64)):
    X, y = [], []
    gestures = sorted(os.listdir(data_dir))

    for label, gesture in enumerate(gestures):
        gesture_path = os.path.join(data_dir, gesture)
        if not os.path.isdir(gesture_path):
            continue
        for file in os.listdir(gesture_path):
            if not file.endswith(".png"):
                continue
            img_path = os.path.join(gesture_path, file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, image_size)
                X.append(img)
                y.append(label)

    if len(X) == 0:
        raise ValueError("No images loaded. Check if dataset path is correct and contains PNG images.")

    X = np.array(X).reshape(-1, image_size[0], image_size[1], 1) / 255.0
    y = np.array(y)
    return train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test = load_data()
print("Data loaded successfully. Shapes:")
print(X_train.shape, y_train.shape)

Using data directory: .
Data loaded successfully. Shapes:
(1, 64, 64, 1) (1,)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np

# Assuming y_train and y_test are already defined
# First, determine the actual number of classes in your data
num_classes = max(np.max(y_train), np.max(y_test)) + 1  # +1 because classes are zero-indexed

# Convert to categorical with the correct number of classes
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_test_cat = to_categorical(y_test, num_classes=num_classes)

# Print shapes to verify
print(f"y_train_cat shape: {y_train_cat.shape}")
print(f"y_test_cat shape: {y_test_cat.shape}")

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    # Use the actual number of classes from your data
    Dense(num_classes, activation='softmax')  # This should match the number of classes in y_train_cat
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint = ModelCheckpoint('gesture_model.h5', save_best_only=True)

# Assuming X_train and X_test are properly preprocessed
model.fit(X_train, y_train_cat, validation_data=(X_test, y_test_cat), epochs=10, batch_size=64, callbacks=[checkpoint])

y_train_cat shape: (1, 5)
y_test_cat shape: (1, 5)


C:\Users\Mamba\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.0000e+00 - loss: 1.7889

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0000e+00 - loss: 1.7889 - val_accuracy: 0.0000e+00 - val_loss: 2.2424
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - accuracy: 1.0000 - loss: 0.9141 - val_accuracy: 0.0000e+00 - val_loss: 3.4173
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step - accuracy: 1.0000 - loss: 0.0730 - val_accuracy: 0.0000e+00 - val_loss: 5.4573
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - accuracy: 1.0000 - loss: 0.1248 - val_accuracy: 0.0000e+00 - val_loss: 8.4201
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - accuracy: 1.0000 - loss: 0.0018 - val_accuracy: 0.0000e+00 - val_loss: 11.3472
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step - accuracy: 1.0000 - loss: 0.0021 - val_accuracy: 0.0000e+00 - val_loss: 14.2598
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step - accuracy: 1.0000 - loss: 2.2411e-05 - val_accuracy: 0.0000e+00 - val_loss: 17.0919
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - accuracy: 1.0000 - loss: 1.3471e-05 - val

In [5]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Load model weights
model.load_weights('gesture_model.h5')

# Generate predictions
predictions = model.predict(X_test)
pred_labels = np.argmax(predictions, axis=1)

# Print classification report with zero_division parameter
print("Classification Report:")
print(classification_report(y_test, pred_labels, zero_division=0))

print("Confusion Matrix:")
print(confusion_matrix(y_test, pred_labels))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
Classification Report:
              precision    recall  f1-score   support

           3       0.00      0.00      0.00       0.0
           4       0.00      0.00      0.00       1.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0

Confusion Matrix:
[[0 0]
 [1 0]]
